# Fase 2 – Manifestos existentes
Notebook dedicado a ler, inspecionar e harmonizar os manifestos anteriores (Atividade 2.2).

## Escopo da Atividade 2.2
- confirmar caminhos reais dos manifestos anteriores;
- inspecionar as colunas disponíveis em cada fonte;
- definir um conjunto mínimo harmonizado (`fonte_manifesto`, `pasta_raiz`, `path_local_base`, `descricao_manifesto`, `qtd_arquivos_reportada`, `observacoes_manifesto`);
- gerar `dados/manifestos_inicial/manifest_intermediario.csv` e expor amostras para validação.


In [1]:
from __future__ import annotations
from pathlib import Path
from typing import Dict, List
import pandas as pd

In [2]:
def find_fase1_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if candidate.name == 'fase_1_diagnostico':
            return candidate
    raise RuntimeError('Não encontrei a pasta fase_1_diagnostico a partir deste notebook')

CURRENT_DIR = Path.cwd().resolve()
FASE1_ROOT = find_fase1_root(CURRENT_DIR)
MANIFEST_INDEX_PATH = FASE1_ROOT / 'dados' / 'manifestos_initial' / 'manifest_index.csv'
INDEX_RESUMO_PATH = FASE1_ROOT / 'dados' / 'analise' / 'index_resumo_por_pasta.csv'
OUTPUT_PATH = FASE1_ROOT / 'dados' / 'manifestos_inicial' / 'manifest_intermediario.csv'

FASE1_ROOT, MANIFEST_INDEX_PATH.exists(), INDEX_RESUMO_PATH.exists(), OUTPUT_PATH.parent.exists()

(PosixPath('/home/wilson/Maringa/fase_1_diagnostico'), True, True, True)

In [3]:
manifest_index_df = pd.read_csv(MANIFEST_INDEX_PATH)
index_resumo_df = pd.read_csv(INDEX_RESUMO_PATH)
manifest_index_df.head(3), index_resumo_df.head(3)

(      file_id                                relative_path  \
 0  20c2f8fa62  dados_iniciais/Corridas/2019_F2_Corrida.csv   
 1  6a4d7a81a0  dados_iniciais/Corridas/2025_F3_Corrida.csv   
 2  5a4f92dc42  dados_iniciais/Corridas/2022_F5_Corrida.csv   
 
              file_name   ext  size_mb        last_modified type_guess  \
 0  2019_F2_Corrida.csv  .csv    0.541  2025-04-30T10:36:54        csv   
 1  2025_F3_Corrida.csv  .csv    0.209  2025-04-30T10:40:00        csv   
 2  2022_F5_Corrida.csv  .csv    0.858  2025-04-30T10:38:50        csv   
 
   forno_guess variable_guess  n_cols has_timestamp timestamp_column  \
 0         NaN        Corrida    34.0           yes        Data_Base   
 1         NaN        Corrida    34.0           yes        Data_Base   
 2         NaN        Corrida    34.0           yes        Data_Base   
 
   encoding_used  notes  
 0         utf-8    NaN  
 1         utf-8    NaN  
 2         utf-8    NaN  ,
     path_relativo     base_name ext_upper  n_files  

In [4]:
{
    'manifest_index_cols': manifest_index_df.columns.tolist(),
    'index_resumo_cols': index_resumo_df.columns.tolist(),
    'manifest_index_rows': len(manifest_index_df),
    'index_resumo_rows': len(index_resumo_df)
}

{'manifest_index_cols': ['file_id',
  'relative_path',
  'file_name',
  'ext',
  'size_mb',
  'last_modified',
  'type_guess',
  'forno_guess',
  'variable_guess',
  'n_cols',
  'has_timestamp',
  'timestamp_column',
  'encoding_used',
  'notes'],
 'index_resumo_cols': ['path_relativo',
  'base_name',
  'ext_upper',
  'n_files',
  'variantes_csv',
  'descricao_compacta'],
 'manifest_index_rows': 131,
 'index_resumo_rows': 36}

In [7]:
def normalize_manifest_index(df: pd.DataFrame) -> pd.DataFrame:
    rel_path = df['relative_path'].astype(str).str.replace('\\\\', '/', regex=False).str.replace('\\', '/', regex=False)
    pasta_raiz = rel_path.str.split('/').apply(
        lambda parts: parts[1] if parts and parts[0] == 'dados_iniciais' and len(parts) > 1 else (parts[0] if parts else '')
    )
    path_local_base = rel_path.apply(
        lambda rp: f"dados/{rp}" if not rp.startswith('dados/') else rp
    )

    def pick_description(row):
        for candidate in ('notes', 'variable_guess', 'type_guess'):
            value = row.get(candidate)
            if isinstance(value, str) and value.strip():
                return value.strip()
        return None

    def build_observacoes(row):
        fragments = []
        if isinstance(row.get('type_guess'), str) and row['type_guess']:
            fragments.append(f"type={row['type_guess']}")
        if isinstance(row.get('forno_guess'), str) and row['forno_guess']:
            fragments.append(f"forno={row['forno_guess']}")
        if isinstance(row.get('variable_guess'), str) and row['variable_guess']:
            fragments.append(f"variavel={row['variable_guess']}")
        if isinstance(row.get('notes'), str) and row['notes'].strip():
            fragments.append(row['notes'].strip())
        return '; '.join(fragments) if fragments else None

    normalized = pd.DataFrame({
        'fonte_manifesto': 'manifest_index',
        'pasta_raiz': pasta_raiz,
        'path_local_base': path_local_base,
        'descricao_manifesto': df.apply(pick_description, axis=1),
        'qtd_arquivos_reportada': 1,
        'observacoes_manifesto': df.apply(build_observacoes, axis=1)
    })
    return normalized

In [8]:
normalized_manifest_index = normalize_manifest_index(manifest_index_df)
normalized_index_resumo = normalize_index_resumo(index_resumo_df)
manifest_intermediario = pd.concat(
    [normalized_manifest_index, normalized_index_resumo],
    ignore_index=True
).sort_values(['pasta_raiz', 'fonte_manifesto', 'path_local_base']).reset_index(drop=True)
manifest_intermediario.head(5)

,fonte_manifesto,pasta_raiz,path_local_base,descricao_manifesto,qtd_arquivos_reportada,observacoes_manifesto
0,index_resumo_por_pasta,Consumo Fornos,dados/dados_iniciais/Consumo Fornos,"2018_CONSUMO; F5, F4, F3, F2; CSV",4,"F5, F4, F3, F2"
1,index_resumo_por_pasta,Consumo Fornos,dados/dados_iniciais/Consumo Fornos,"2019_CONSUMO; F5, F4, F3, F2, F1; CSV",5,"F5, F4, F3, F2, F1"
2,index_resumo_por_pasta,Consumo Fornos,dados/dados_iniciais/Consumo Fornos,"2020_CONSUMO; F5, F4, F3, F2, F1; CSV",5,"F5, F4, F3, F2, F1"
3,index_resumo_por_pasta,Consumo Fornos,dados/dados_iniciais/Consumo Fornos,"2021_CONSUMO; F5, F4, F3, F2, F1; CSV",5,"F5, F4, F3, F2, F1"
4,index_resumo_por_pasta,Consumo Fornos,dados/dados_iniciais/Consumo Fornos,"2022_CONSUMO; F5, F4, F3, F2, F1; CSV",5,"F5, F4, F3, F2, F1"


In [9]:
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
manifest_intermediario.to_csv(OUTPUT_PATH, index=False)
OUTPUT_PATH, len(manifest_intermediario)

(PosixPath('/home/wilson/Maringa/fase_1_diagnostico/dados/manifestos_inicial/manifest_intermediario.csv'),
 167)

In [10]:
group_summary = (
    manifest_intermediario
    .groupby(['fonte_manifesto', 'pasta_raiz'])
    .size()
    .unstack(fill_value=0)
    .sort_index()
 )
group_summary

pasta_raiz,Consumo Fornos,Corridas,Dicionário,Dicionário,Eletrodo,Informações Diária,Informações Diária,Supervisorio Forno 4,Supervisorio Forno 5
fonte_manifesto,,,,,,,,,
index_resumo_por_pasta,8,8,0,7,1,0,8,2,2
manifest_index,39,40,7,0,1,40,0,2,2


In [12]:
samples = (
    manifest_intermediario
    .groupby('pasta_raiz', group_keys=False)
    .head(1)
    .reset_index(drop=True)
 )
samples

,fonte_manifesto,pasta_raiz,path_local_base,descricao_manifesto,qtd_arquivos_reportada,observacoes_manifesto
0,index_resumo_por_pasta,Consumo Fornos,dados/dados_iniciais/Consumo Fornos,"2018_CONSUMO; F5, F4, F3, F2; CSV",4,"F5, F4, F3, F2"
1,index_resumo_por_pasta,Corridas,dados/dados_iniciais/Corridas,"2018_CORRIDA; F5, F4, F3, F2, F1; CSV",5,"F5, F4, F3, F2, F1"
2,manifest_index,Dicionário,dados/dados_iniciais/Dicionário/Dic_Consumo.xlsx,Consumo,1,type=xlsx; variavel=Consumo
3,index_resumo_por_pasta,Dicionário,dados/dados_iniciais/Dicionário,DIC_CONSUMO; XLSX,1,None
4,index_resumo_por_pasta,Eletrodo,dados/dados_iniciais/Eletrodo,MEDIÇÃO_ELETRODO; CSV,1,None
5,manifest_index,Informações Diária,dados/dados_iniciais/Informações Diária/201...,csv,1,type=csv
6,index_resumo_por_pasta,Informações Diária,dados/dados_iniciais/Informações Diária,"2018_INF_DIARIO; F5, F4, F3, F2, F1; CSV",5,"F5, F4, F3, F2, F1"
7,index_resumo_por_pasta,Supervisorio Forno 4,dados/dados_iniciais/Supervisorio Forno 4,2024_1S; F4; CSV,1,F4
8,index_resumo_por_pasta,Supervisorio Forno 5,dados/dados_iniciais/Supervisorio Forno 5,SUPERVISORIO_MINUTO; F5; CSV,1,F5
